Import packages

In [8]:
from utils import *
import numpy as np
import torch
from valda.valuation import DataValuation

Initialize the parameters

In [9]:
train_size = 1000
train_start = 0
train_size_batch = 200
dev_size = 200
test_size = 300
np.random.seed(0)
torch.manual_seed(0)

Load the MNIST dataset and split it into training and test sets.

In [10]:
X_train, y_train_clean, X_dev, y_dev, X_test, y_test = load_data(train_size, dev_size, test_size)
y_train, flip_indices = flip_labels(y_train_clean, flip_fraction=0.3)

Divide the train set into batches and run the valuation.

In [11]:
X_train_batch = X_train[train_start:train_start+train_size_batch]
y_train_batch = y_train[train_start:train_start+train_size_batch]

In [12]:
# Define a DataValuation instance
dv = DataValuation(X_train_batch, y_train_batch, X_dev, y_dev)

# Run with the default setting:
# 1. Logistic regression as the classifier
# 2. Prediction accuracy as the value measurement
vals_tmc = dv.estimate(method="tmc-shapley")

100%|██████████| 500/500 [22:42<00:00,  2.72s/it]


Get the indices of the data points with values < 0

In [13]:
min_indices_batch = [key for key, value in vals_tmc.items() if value <= 0]
min_indices_batch = np.array(min_indices_batch) + train_start

Append the indices to a file

In [14]:
if not os.path.exists('./data'):
    os.makedirs('./data')

file_path = f'./data/min_indices_{str(train_start).zfill(3)}-{str(train_start+train_size_batch-1).zfill(3)}_{dev_size}.npy'
np.save(file_path, min_indices_batch)